# 基于Huggingface Peft对比不同微调算法性能

在这篇文章当中，我们将会对各类微调算法有一个初步的认知（手撕还在后面），由于目前微调算法众多让人眼花缭乱，因此我们主要挑选工业界和学界讨论比较多的几个算法


## 什么是peft？

PEFT（参数高效微调）是一种针对预训练模型（尤其是大语言模型）的库，能够有效的将预训练模型适配到各种下游应用，在仅调整少量模型参数的同时，能够获得和原始模型基本一致的效果，降低了计算成本和存储成本的同时，也保留了模型的泛化能力。目前Huggingface将[peft](https://github.com/huggingface/peft)框架与其他库如tranformers包等继承提供了一种简单的方法来训练大模型

目前这个peft框架支持了很多算法，包括但不限于Prefix Tuning、Prompt Tuning、Adapter、LoRA等，可以实现

> 在[peft模型列表](https://hugging-face.cn/docs/peft/index)中可以查询模型是否能够使用peft框架训练

在开始前，让我们先下载好相关的包，在设计的时候我们使用了是python=3.12的版本，首先需要安装pytorch(如果已经安装则跳过），通过`nvidia-smi`查看cuda版本

In [2]:
!nvidia-smi

Tue Jul  1 06:34:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:01:00.0 Off |                  Off |
| 62%   71C    P2            444W /  450W |   19413MiB /  24564MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

此外，还需要安装peft和其他相关包

In [ ]:
!pip install peft transformers datasets accelerate

如果peft想要通过源代码安装，可以按照下面的方式

In [ ]:
!pip install git+https://github.com/huggingface/peft

### 数据加载

</br>首先要对加载的几个库进行说明：

</br>1. datasets: 这里我们加载的数据是从huggingface上加载的数据集，ds并不是我们直接能用的数据，直接显示ds数据会得到的是数据集本身的信息
</br>
    ```python
    DatasetDict({
        train: Dataset({
            features: ['instruction_zh', 'input_zh', 'output_zh', 'instruction', 'input', 'output'],
            num_rows: 52049
        })
    })
    ```
</br>因此我们需要对数据集进行提取，提取方法有两个，一个是在load_dataset当中直接使用split，第二个则是索引ds['train']来获得，之后也可以使用train_test_split来划分验证集和测试集

</br>2. 这里我们用的是`AutoModelForCausalLM`类，这个类是因果模型，因果模型主要是用于预测token序列中的下一个token，而模型同时也看不到未来的token，GPT-2就是采用的这类模型

</br> 3. `Trainer`是一个训练器，可以方便地用于模型的训练和分布式训练； `TrainingArguments`则是配置训练过程的各种参数，并且能够指定输出路径和日志记录等

</br>4. `DataCollatorForSeq2Seq` 是用于确保seq2seq模型的输入数据中的每个批次的所有序列的长度相同，并生成相应的标签

In [ ]:
from logging import warning
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import random

random.seed(2025)
ds = load_dataset("silk-road/alpaca-data-gpt4-chinese", split = "train[:15000]")
# 
# ds = Dataset.load_from_disk("../data/alpaca-data-gpt4-chinese")


Using the latest cached version of the dataset since silk-road/alpaca-data-gpt4-chinese couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/ky/.cache/huggingface/datasets/silk-road___alpaca-data-gpt4-chinese/default/0.0.0/81a6dfd72f416aff605e7d189bfbbc46a2511fee (last modified on Tue Jul  1 06:37:49 2025).


In [28]:
ds[0]

{'instruction_zh': '给出三个保持健康的小贴士。',
 'input_zh': '',
 'output_zh': '1. 饮食要均衡且富有营养：确保你的餐食包含各种水果、蔬菜、瘦肉、全谷物和健康脂肪。这有助于为身体提供必要的营养，使其发挥最佳功能，并有助于预防慢性疾病。2. 经常参加体育锻炼：锻炼对于保持强壮的骨骼、肌肉和心血管健康至关重要。每周至少要进行150分钟的中等有氧运动或75分钟的剧烈运动。3. 获得足够的睡眠：获得足够的高质量睡眠对身体和心理健康至关重要。它有助于调节情绪，提高认知功能，并支持健康的生长和免疫功能。每晚睡眠目标为7-9小时。',
 'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps t

### 数据预处理

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", local_files_only=True)


首先我们定义一个初步的预处理函数`process_data`，这个预处理函数的目的是去为了更方便的对结果进行分词

In [8]:
def process_data(example):
    """
    这个函数的目的是将数据转换为模型可以接受的格式，并返回input_ids, attention_mask, labels
    """
    input_ids, attention_mask, labels = [], [], []
    MAX_LENGTH = 512 # 设置最大长度
    instruction = tokenizer("\n".join(["Human:", example['instruction_zh'], example['input_zh']]).strip() + "\n\nAssistant:") 
    # 将instruction和input拼接起来，并添加一个Assistant，之所以这么做，是因为采用human + assistant的格式，可以使得模型明确角色类型
    response = tokenizer(example['output_zh'] + tokenizer.eos_token) # 将output和eos_token拼接起来
    input_ids = instruction['input_ids'] + response['input_ids']
    attention_mask = instruction['attention_mask'] + response['attention_mask'] 
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] # 这里使用-100来表示不需要计算loss的token，-100在损失函数中会被忽略
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels}
    

我们使用 `Dataset` 类的map方法将预处理函数应用到整个数据集

In [21]:
data_tokenization = ds.map(process_data, remove_columns=list(ds.column_names))
tokenizer.decode(data_tokenization[0]['input_ids'])
tokenizer.decode(list(filter(lambda x: x != -100, data_tokenization[0]['labels'])))


'1. 饮食要均衡且富有营养：确保你的餐食包含各种水果、蔬菜、瘦肉、全谷物和健康脂肪。这有助于为身体提供必要的营养，使其发挥最佳功能，并有助于预防慢性疾病。2. 经常参加体育锻炼：锻炼对于保持强壮的骨骼、肌肉和心血管健康至关重要。每周至少要进行150分钟的中等有氧运动或75分钟的剧烈运动。3. 获得足够的睡眠：获得足够的高质量睡眠对身体和心理健康至关重要。它有助于调节情绪，提高认知功能，并支持健康的生长和免疫功能。每晚睡眠目标为7-9小时。<|im_end|>'

## 微调方法比较

In [10]:
import pandas as pd
from peft import (
    LoraConfig,
    PrefixTuningConfig,
    PromptTuningConfig,
    PromptEncoderConfig,
    AdaLoraConfig,
    TaskType,
    PromptTuningInit,
    PromptEncoderReparameterizationType,
    get_peft_model
)

def choose_sft_method(sft_method: str) -> dict:
    if sft_method == "lora":
        # 使用lora方法
        lora_config = LoraConfig(
            task_type = TaskType.CAUSAL_LM,
            r = 8,
            lora_alpha = 32,
            lora_dropout = 0.01,
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
            # modules_to_save=["word_embeddings"]
        )
        return lora_config
    elif sft_method == "prefix-tuning":
        # 使用prefix方法
        prefix_config = PrefixTuningConfig(
            task_type = TaskType.CAUSAL_LM,
            num_virtual_tokens = 10,
            prefix_projection = True
        )
        return prefix_config
    elif sft_method == "prompt-tuning":
        # 使用prompt方法
        prompt_config = PromptTuningConfig(
            task_type = TaskType.CAUSAL_LM,
            prompt_tuning_init = PromptTuningInit.TEXT,
            prompt_tuning_init_text = "下面是一个问题和相应的回答",
            num_virtual_tokens = len(tokenizer("下面是一个问题和相应的回答")["input_ids"]),
            tokenizer_name_or_path = "Qwen/Qwen3-0.6B"
        )
        return prompt_config
    elif sft_method == "p-tuning":
        # 使用p-tuning方法
        p_tuning_config = PromptEncoderConfig(
            task_type = TaskType.CAUSAL_LM,
            num_virtual_tokens = 10,
            encoder_reparameterization_type = PromptEncoderReparameterizationType.MLP,
            encoder_dropout = 0.1, 
            encoder_num_layers = 8,
            encoder_hidden_size = 1024
        )
        return p_tuning_config
    elif sft_method == "adapter":
        # 使用adapter方法，这里需要注意的是adapterlora需要确定target_modules,而target_modules需要根据模型结构来确定，具体请使用model.named_modules输出模型结构，也可以使用默认的target_modules
        adapter_config = AdaLoraConfig(
            task_type = TaskType.CAUSAL_LM,
            r = 8,
            lora_alpha = 32,
            lora_dropout = 0.1,
            target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"], # 如果只使用query & value，意思就是仅对注意力层的q v 矩阵进行训练
            total_step = 5000
            
        )
        return adapter_config
    else:
        raise ValueError(f"Invalid SFT method: {sft_method}")

为了更好的展示不同微调算法的区别，我们将可训练参数的结果可视化成表格，结果显示prompt-tuning的参数使用最少

In [ ]:
methods = ["lora", "prefix-tuning", "prompt-tuning", "p-tuning", "adapter"]
data = []

import io
import sys
from contextlib import redirect_stdout
import re

for method in methods:
    base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", local_files_only=True)
    config = choose_sft_method(method)
    peft_model = get_peft_model(base_model, peft_config=config)
    
    f = io.StringIO()
    with redirect_stdout(f):
        peft_model.print_trainable_parameters()
    info = f.getvalue().strip()
    
    match = re.search(r'trainable params: ([\d,]+) \|\| all params: ([\d,]+) \|\| trainable%: ([\d\.]+)', info)
    if match:
        trainable = match.group(1)
        all_params = match.group(2)
        trainable_pct = match.group(3)
        
        data.append({
            "方法": method,
            "可训练参数": trainable,
            "总参数": all_params,
            "可训练比例(%)": trainable_pct
        })


df = pd.DataFrame(data)


In [14]:
df

,方法,可训练参数,总参数,可训练比例(%)
0,lora,"2,293,760","598,343,680",0.3834
1,prefix-tuning,"14,976,512","611,026,432",2.4510
2,prompt-tuning,"6,144","596,056,064",0.0010
3,p-tuning,"3,159,040","599,208,960",0.5272
4,adapter,"3,441,984","599,492,016",0.5742


### 模型训练

接下来我们对每个微调方法进行实际的训练看一下效果如何

#### lora

In [22]:
####### 构建训练器 #######

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", local_files_only=True)
args = TrainingArguments(
    output_dir="./model/lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=20,
)

trainer = Trainer(
    model=model,
    args=args,
    processing_class = tokenizer,
    train_dataset = data_tokenization,
    data_collator = DataCollatorForSeq2Seq(tokenizer, padding = True)
)


In [ ]:
import torch
torch.cuda.empty_cache() # 防止出现显存不足的问题
trainer.train()

In [16]:
### 测试推理 #####
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("如何保持健康", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True)


'Human: 如何保持健康\n\nAssistant: 保持健康是一个长期的过程，需要科学的方法和持续的努力。以下是一些常见的建议和方法，帮助你保持健康：\n\n1. **均衡饮食**：保持营养均衡，摄入足够的蛋白质、碳水化合物、脂肪和维生素。多吃蔬果，少食多餐。\n\n2. **规律运动**：每周至少150分钟中等强度运动，如快走、游泳、骑自行车等。运动有助于增强心肺功能，改善睡眠，增强免疫力。\n\n3. **充足睡眠**：保持每天7-9小时的高质量睡眠。睡眠不足'